### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [ ]:
appName = "Constructors Ingestion"
spark = create_spark_context(appName, warehouse_location)

In [ ]:
results_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/results")

### Step 2 - Rename and drop columns, and add new columns

In [ ]:
results_column_mapping = {
    "resultId": "result_id",
    "raceId": "race_id",
    "driverId": "driver_id",
    "constructorId": "constructor_id",
    "positionText": "position_text",
    "positionOrder": "position_order",
    "fastestLap": "fastest_lap",
    "fastestLapTime": "fastest_lap_time",
    "fastestLapSpeed": "fastest_lap_speed"
}

results_df = rename_columns(results_df, results_column_mapping)

In [ ]:
columns_to_drop = ['statusId']

results_df = drop_columns(results_df, columns_to_drop)

In [ ]:
results_df = add_ingestion_date(results_df)

### Step 3 - Write a data to a Datalake as a Parquet

In [ ]:
results_df.write.mode("overwrite").partitionBy("race_id").parquet(f"{processed_folder_path}/results")